In [16]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet34
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset, read_pred_csv
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

from torch import Tensor
from collections import OrderedDict, defaultdict
import copy

import os

## Prepare Data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [17]:
DEBUG = False

# training cfg
validation_cfg = {
    
    'format_version': 4,
    
     ## Model options
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    ## Input raster parameters
    'raster_params': {
        
        'raster_size': [224, 224], # raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.
        'pixel_size': [0.5, 0.5], # From 0 to 1 per axis, [0.5,0.5] would show the ego centered in the image.
        'ego_center': [0.25, 0.5],
        'map_type': "py_semantic",
        
        # the keys are relative to the dataset environment variable
        'satellite_map_key': "aerial_map/aerial_map.png",
        'semantic_map_key': "semantic_map/semantic_map.pb",
        'dataset_meta_key': "meta.json",

        # e.g. 0.0 include every obstacle, 0.5 show those obstacles with >0.5 probability of being
        # one of the classes we care about (cars, bikes, peds, etc.), >=1.0 filter all other agents.
        'filter_agents_threshold': 0.5
    },

    ## Data loader options
    'valid_data_loader': {
        'key': "scenes/validate_chopped_100/validate.zarr",
        'batch_size': 1,
        'shuffle': False,
        'num_workers': 0
    },

    ## Valid params
    'valid_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 10 if DEBUG else 1000
    }
}

common_cfg = {
    'seed': 500,
    'output_dir': './subs/',
    'epoch': 2,
    'train_step': 5 if DEBUG else 500,
    'valid_step': 5 if DEBUG else 50,
    'train_max': 12,
    'learning_rate': 1e-3
}


In [18]:
OUTPUT_DIR = common_cfg['output_dir']
INPUT_ROOT = Path('/home/knikaido/work/Lyft/data/')
DATA_DIR = INPUT_ROOT / 'lyft-motion-prediction-autonomous-vehicles/'

In [19]:
pred_path1 = f"{OUTPUT_DIR}sub_ansamble.csv"
pred_path2 = f"{OUTPUT_DIR}sub_ansamble_22_28.csv"

In [20]:
pred_pathes = [pred_path1, pred_path2]
outputs = []
for path in pred_pathes:
    output = pd.read_csv(path)
    output.sort_values(['timestamp', 'track_id'], inplace=True)
    output.reset_index(inplace=True, drop=True)
    outputs.append(output)


In [21]:
t_predicts = []
t_confs = []

In [22]:
for i in tqdm(range(71122)):
    predicts = []
    confs = []
    confs.append(outputs[0].loc[i, 'conf_0'])
    confs.append(outputs[0].loc[i, 'conf_1'])
    confs.append(outputs[0].loc[i, 'conf_2'])
    predicts.append(outputs[0].loc[i, 'coord_x00':'coord_y049'].values)
    predicts.append(outputs[0].loc[i, 'coord_x10':'coord_y149'].values)
    predicts.append(outputs[0].loc[i, 'coord_x20':'coord_y249'].values)
    confs.append(outputs[1].loc[i, 'conf_0'])
    confs.append(outputs[1].loc[i, 'conf_1'])
    confs.append(outputs[1].loc[i, 'conf_2'])
    predicts.append(outputs[1].loc[i, 'coord_x00':'coord_y049'].values)
    predicts.append(outputs[1].loc[i, 'coord_x10':'coord_y149'].values)
    predicts.append(outputs[1].loc[i, 'coord_x20':'coord_y249'].values)

    t_predicts.append(predicts)
    t_confs.append(confs)

100%|██████████| 71122/71122 [03:03<00:00, 387.09it/s]


In [23]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [24]:
def calc_distance(coord1, coord2):
    coord1 = coord1.reshape(-1, 2)
    coord2= coord2.reshape(-1, 2)
    distance = 0
    cos = 0
    for i in range(len(coord1)):
        cos += cos_sim(coord2[i], coord1[i])
        distance += pow(coord2[i, 0] - coord1[i, 0], 2) + pow(coord2[i, 1] - coord1[i, 1], 2)
        
    return cos / len(coord1), np.sqrt(distance)
    

In [25]:
def decide_preds(tmp_predicts, tmp_conf, thre=15):
    
    
    predicts = copy.deepcopy(tmp_predicts)
    confs = copy.deepcopy(tmp_conf)
    
    ans_preds = []
    ans_confs = []
    
    for i in range(3):
        ind = np.argmax(confs)
        pre1 = predicts[ind]
        ans_preds.append(pre1)
        ans_confs.append(confs[ind])
        predicts.pop(ind)
        confs.pop(ind)

        predicts_ = []
        confs_ = []
        for i in range(len(predicts)):
            angle, dist = calc_distance(pre1, np.array(predicts[i]))
            if(dist > thre):
                predicts_.append(predicts[i])
                confs_.append(confs[i])
                
        if(len(confs_)<3):
            return tmp_predicts[0:3], tmp_conf[0:3] / np.sum(tmp_conf[0:3])
            
        predicts = predicts_
        confs = confs_
        
    ans_confs = ans_confs / np.sum(ans_confs)
    return ans_preds, ans_confs
        

In [26]:
for i in range(len(t_confs)):
    for j in range(3):
#         t_confs[i][j] = t_confs[i][j] * 2  
        t_confs[i][j] = t_confs[i][j] * 10 * (26.2 / 21.2)  

In [27]:
ans_output = pd.read_csv(str(OUTPUT_DIR)+'sub1010.csv')
ans_output.sort_values(['timestamp', 'track_id'], inplace=True)
ans_output.reset_index(inplace=True, drop=True)
ans_conf = ans_output.loc[:, 'conf_0':'conf_2']
ans_coord= ans_output.loc[:, 'coord_x00':]

In [28]:
for j in tqdm(range(len(ans_output))):
    prepre, concon = decide_preds(t_predicts[j], t_confs[j])

    ans_conf.iloc[j] = concon
    ans_coord.iloc[j] = np.array(prepre).reshape(-1)


 60%|██████    | 42794/71122 [03:34<02:28, 190.20it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
100%|██████████| 71122/71122 [05:56<00:00, 199.52it/s]


In [29]:
ans_output.loc[:, 'conf_0':'conf_2'] = ans_conf
ans_output.loc[:, 'coord_x00':] = ans_coord

In [30]:
ans_output_ = pd.read_csv(str(OUTPUT_DIR)+'sub1010.csv')
ans_output_.sort_values(['timestamp', 'track_id'], inplace=True)
ans_output.index = ans_output_.index
ans_output.sort_index(inplace=True)
ans_output.to_csv(str(OUTPUT_DIR)+'sub_ansamble_ble.csv', index=False)
